In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob

import pandas as pd

In [3]:
big_data = pd.DataFrame()
files = glob.iglob('backtests/binance/' + '**/backtest_result.txt', recursive=True)
i = 0
for filename in files:
    #print(f'parsing : {filename}')
    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': 'C:\\AgodaGit\\passivbot\\'+filename}
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value
                if key == 'Total gain percentage':
                    d['tg_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain percentage':
                    d['adg_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy percentage':
                    d['bnkr_%'] = float(value.replace('%', '').strip())
                if key == 'Long':
                    d['long'] = bool(value)
                if key == 'Short':
                    d['short'] = bool(value)
                if key == 'Lowest equity/balance ratio':
                    d['ebr'] = float(value.replace('%', '').strip())

        big_data = big_data.append(d, ignore_index=True)

big_data

C:\Users\skhan\AppData\Local\Temp/ipykernel_384704/3000735749.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_384704/3000735749.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_384704/3000735749.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_384704/3000735749.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_

,file,v,s,tg_%,adg_%,bnkr_%,ebr,long,short
0,C:\AgodaGit\passivbot\backtests/binance\ADAUSD...,5.3,ADAUSDT,5.860,0.03370,100.0,0.976541,True,NaN
1,C:\AgodaGit\passivbot\backtests/binance\ADAUSD...,5.3,ADAUSDT,10.360,0.05600,100.0,0.887775,True,True
2,C:\AgodaGit\passivbot\backtests/binance\ADAUSD...,5.3,ADAUSDT,5.592,0.03090,100.0,0.886605,NaN,True
3,C:\AgodaGit\passivbot\backtests/binance\ADAUSD...,5.3,ADAUSDT,5.252,0.03030,100.0,0.976716,True,NaN
4,C:\AgodaGit\passivbot\backtests/binance\ADAUSD...,5.3,ADAUSDT,10.420,0.05640,100.0,0.889643,True,True
...,...,...,...,...,...,...,...,...,...
103,C:\AgodaGit\passivbot\backtests/binance\XRPUSD...,5.3,XRPUSDT,5.463,0.02980,100.0,0.968618,True,True
104,C:\AgodaGit\passivbot\backtests/binance\XRPUSD...,5.3,XRPUSDT,1.525,0.00838,100.0,0.968618,NaN,True
105,C:\AgodaGit\passivbot\backtests/binance\XTZUSD...,5.3,XTZUSDT,5.246,0.02990,100.0,0.972171,True,NaN
106,C:\AgodaGit\passivbot\backtests/binance\XTZUSD...,5.3,XTZUSDT,6.165,0.03400,100.0,0.847562,True,True


Now filter the dataframe

In [5]:
allowed_symbols = ["ETHUSDT", "XRPUSDT", "1000SHIBUSDT", "DOGEUSDT", "ADAUSDT",
            "BNBUSDT", "MATICUSDT", "DOTUSDT", "SANDUSDT", "FTMUSDT",
            "LTCUSDT", "LINKUSDT", "MANAUSDT", "ETCUSDT", "EOSUSDT",
            "ATOMUSDT", "FILUSDT", "ICPUSDT", "ALICEUSDT", "LRCUSDT",
            "BCHUSDT", "SUSHIUSDT", "XTZUSDT", "THETAUSDT", "CRVUSDT",
            "ALGOUSDT", "CHZUSDT", "TRXUSDT", "VETUSDT", "CHRUSDT",
            "ONEUSDT", "OMGUSDT", "XLMUSDT", "RUNEUSDT", "ENJUSDT",
            "CELRUSDT", "SXPUSDT", "1INCHUSDT", "SRMUSDT", "COTIUSDT",
            "ZECUSDT", "QTUMUSDT", "GRTUSDT", "DENTUSDT", "SFPUSDT",
            "STORJUSDT", "IOTAUSDT", "HOTUSDT", "NEOUSDT", "COMPUSDT",
            "LINAUSDT", "KAVAUSDT", "BATUSDT", "WAVESUSDT", "IOSTUSDT",
            "ALPHAUSDT", "RLCUSDT", "HBARUSDT", "DODOUSDT", "XMRUSDT",
            "SNXUSDT", "ANKRUSDT", "DASHUSDT", "REEFUSDT", "ZENUSDT",
            "CVCUSDT", "RENUSDT", "ICXUSDT", "RSRUSDT", "SKLUSDT",
            "NKNUSDT", "BELUSDT", "ONTUSDT", "OGNUSDT", "BLZUSDT",
            "MTLUSDT", "CTKUSDT", "RVNUSDT", "LITUSDT", "MKRUSDT",
            "OCEANUSDT", "ZILUSDT", "UNFIUSDT", "BANDUSDT", "YFIIUSDT",
            "TOMOUSDT", "TRBUSDT", "KNCUSDT", "STMXUSDT", "ZRXUSDT",
            "SCUSDT", "AKROUSDT", "XEMUSDT", "FLMUSDT", "BALUSDT",
            "BTSUSDT", "DGBUSDT", "DEFIUSDT"]

#allowed_symbols = ['ETHUSDT']

df = big_data[(big_data.s.isin(allowed_symbols)) &
              (big_data['bnkr_%'] > 95.0) &
              (big_data['tg_%'] > 0) &
              (big_data['v'] == '5.3') ]
df

,file,v,s,tg_%,adg_%,bnkr_%,ebr,long,short
0,C:\AgodaGit\passivbot\backtests/binance\ADAUSD...,5.3,ADAUSDT,5.860,0.03370,100.0,0.976541,True,NaN
1,C:\AgodaGit\passivbot\backtests/binance\ADAUSD...,5.3,ADAUSDT,10.360,0.05600,100.0,0.887775,True,True
2,C:\AgodaGit\passivbot\backtests/binance\ADAUSD...,5.3,ADAUSDT,5.592,0.03090,100.0,0.886605,NaN,True
3,C:\AgodaGit\passivbot\backtests/binance\ADAUSD...,5.3,ADAUSDT,5.252,0.03030,100.0,0.976716,True,NaN
4,C:\AgodaGit\passivbot\backtests/binance\ADAUSD...,5.3,ADAUSDT,10.420,0.05640,100.0,0.889643,True,True
...,...,...,...,...,...,...,...,...,...
103,C:\AgodaGit\passivbot\backtests/binance\XRPUSD...,5.3,XRPUSDT,5.463,0.02980,100.0,0.968618,True,True
104,C:\AgodaGit\passivbot\backtests/binance\XRPUSD...,5.3,XRPUSDT,1.525,0.00838,100.0,0.968618,NaN,True
105,C:\AgodaGit\passivbot\backtests/binance\XTZUSD...,5.3,XTZUSDT,5.246,0.02990,100.0,0.972171,True,NaN
106,C:\AgodaGit\passivbot\backtests/binance\XTZUSD...,5.3,XTZUSDT,6.165,0.03400,100.0,0.847562,True,True
